# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from datetime import datetime
from datetime import date
from dateutil import relativedelta

# Load Dataset Awal/Asli

In [ ]:
clickstream = pd.read_csv('/content/drive/MyDrive/Mockup Datasets/click_stream.csv')
customer = pd.read_csv('/content/drive/MyDrive/Mockup Datasets/customer.csv')
product = pd.read_csv('/content/drive/MyDrive/Mockup Datasets/product.csv', on_bad_lines='skip')
transaction = pd.read_csv('/content/drive/MyDrive/Mockup Datasets/transactions.csv')

# Data Exploration & Future Engineering



## Clickstream

In [ ]:
clickstream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri..."
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'}


### One Hot Encoding

In [ ]:
columns_name = ['event_name', 'traffic_source']
one_hot = pd.get_dummies(clickstream[columns_name], columns=columns_name)

In [ ]:
clickstream = pd.concat([clickstream, one_hot], axis=1)
clickstream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0


### Split Dataset

#### Window #1 (1 Apr 21 - 31 Sep 21)

##### Data Train

In [ ]:
clickstream_train_w1 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2021-08-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w1.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
6592374,4fe63050-8b5e-4e03-9328-17c4926761db,CLICK,2021-04-01T00:50:12.168488Z,73431488-4866-49fe-aeed-58bb38dcb60f,WEB,NaN,0,0,0,1,0,0,0,0,0,0,1
7874964,b6b1bb7c-040e-4d8f-a679-8836a70b1ee7,SCROLL,2021-04-01T00:50:32.248551Z,0889d41d-3cfe-4e6d-a87e-494e2d3da4d8,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
11560114,cd7a8e8d-dbdd-4809-823d-d64c5a1f07cf,CLICK,2021-04-01T00:50:32.568581Z,4d100e98-1e40-4abe-9103-28b14e935f9e,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
10025192,d336c29c-e5d9-403d-997d-faaa1f7a10a6,ADD_PROMO,2021-04-01T00:50:32.983766Z,4fbc4a8b-14d1-4412-b010-47b9301dd202,MOBILE,"{'promo_code': 'AZ2022', 'promo_amount': 8095}",1,0,0,0,0,0,0,0,0,1,0
4287234,72337160-dd4c-4051-b531-1b292a94859b,HOMEPAGE,2021-04-01T00:50:52.748343Z,4669398f-0f80-40f5-a483-4ff9e5865440,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258229,e315b4a5-d9af-4865-9e1a-2565ee783e97,CLICK,2021-08-01T00:49:31.901202Z,0e11aded-51fa-4ef6-bac7-aae4649f8b2d,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
1310440,1f298162-2553-4620-9161-c2fc662a7c75,ADD_TO_CART,2021-08-01T00:49:34.313421Z,9d03b69d-3986-4c0d-af6d-a4cdac022512,MOBILE,"{'product_id': 27417, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
416409,36ce8a98-8c93-4969-85bf-d9a4f349e1bd,PROMO_PAGE,2021-08-01T00:49:39.995110Z,c8cae2b8-bb27-4b1f-afe1-bcff5cf92b12,MOBILE,NaN,0,0,0,0,0,0,1,0,0,1,0
161552,70067fe8-e352-4694-853e-e415733e4f7d,SCROLL,2021-08-01T00:49:42.330511Z,dec3ae10-38f4-47d6-8807-ee06d1a2c087,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0


##### Data Test

In [ ]:
clickstream_test_w1 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-08-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2021-10-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w1.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
4381704,172f052d-cf11-4ec0-a02f-5bd175f5df79,SCROLL,2021-08-01T00:50:04.312665Z,ae4c3010-ae78-4e31-b86a-1f672037a9ad,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
6457803,3d769f1e-c83e-4032-a6d8-7c90d13dd075,SCROLL,2021-08-01T00:50:06.366545Z,4e908f67-438d-46ef-8273-da15e3927ecf,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
5637434,788c97b8-7847-4c5e-b4fd-e12e11630899,HOMEPAGE,2021-08-01T00:50:10.106319Z,ebfe9e34-4a5f-4853-9c9e-d867c53fa9a2,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
11082412,dd462fef-5ebf-425e-b8b2-2f2270ebfcc7,HOMEPAGE,2021-08-01T00:50:49.120477Z,a4004b47-dd0b-4f36-b6c4-34e7fc6ef7e9,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
8269915,89c839ce-8274-4ce4-a623-49c99bc2f41b,ADD_TO_CART,2021-08-01T00:51:01.107933Z,916ed690-65df-438a-ac24-41cc93b4ba29,WEB,"{'product_id': 4069, 'quantity': 3, 'item_pric...",0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11605275,8f8e4f2b-d8fc-4c30-b37e-4d964f75222f,SCROLL,2021-10-01T00:49:35.314572Z,ba3d74aa-e41f-4d6d-a319-b4998f2ce495,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
7362167,307413c9-77f0-417a-b19f-f9b4bc302072,PROMO_PAGE,2021-10-01T00:49:35.624312Z,bbc257c1-2c1a-4231-8403-c4d29f129596,MOBILE,NaN,0,0,0,0,0,0,1,0,0,1,0
133095,56f0790e-f934-4b2a-983d-535d80feca24,HOMEPAGE,2021-10-01T00:49:40.669060Z,d299a458-8fe0-4848-afd9-8e4855978c8a,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
6823053,23479745-5cc4-4fb3-9ded-fed93583042c,BOOKING,2021-10-01T00:49:46.518784Z,d3219533-d5f4-4b01-9281-834f6cb0019b,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0


#### Window #2 (1 Jun 21 - 31 Nov 21)

##### Data Train

In [ ]:
clickstream_train_w2 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-06-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2021-10-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w2.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
8106331,d7d48637-be07-45f1-99ea-0e05f918553e,CLICK,2021-06-01T00:50:00.028061Z,fe9dbace-c411-4485-ba14-e9d633a76efd,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
4944829,fbed2295-8449-4d15-b145-26faa750e415,HOMEPAGE,2021-06-01T00:50:06.507237Z,50fa1e33-6e89-42aa-83b2-239db9571e38,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
11464595,6daf311b-e111-44fd-b826-13e9f66bd22d,SEARCH,2021-06-01T00:50:32.101581Z,cc02da44-92eb-4d57-8098-e489afc9cecd,MOBILE,{'search_keywords': 'Thrift'},0,0,0,0,0,0,0,0,1,1,0
941208,7e8a58ff-9959-40f4-b0a6-10bfcf135bf1,HOMEPAGE,2021-06-01T00:51:08.008023Z,98282e01-ff55-4c09-8791-63f149aa0f75,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
371734,aa8ffcb8-c478-4997-a60a-cce3717912ae,SEARCH,2021-06-01T00:51:19.181103Z,a8f511cb-d81d-47d4-a6d4-897af8d1f1e0,MOBILE,{'search_keywords': 'Bekas'},0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11605275,8f8e4f2b-d8fc-4c30-b37e-4d964f75222f,SCROLL,2021-10-01T00:49:35.314572Z,ba3d74aa-e41f-4d6d-a319-b4998f2ce495,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
7362167,307413c9-77f0-417a-b19f-f9b4bc302072,PROMO_PAGE,2021-10-01T00:49:35.624312Z,bbc257c1-2c1a-4231-8403-c4d29f129596,MOBILE,NaN,0,0,0,0,0,0,1,0,0,1,0
133095,56f0790e-f934-4b2a-983d-535d80feca24,HOMEPAGE,2021-10-01T00:49:40.669060Z,d299a458-8fe0-4848-afd9-8e4855978c8a,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
6823053,23479745-5cc4-4fb3-9ded-fed93583042c,BOOKING,2021-10-01T00:49:46.518784Z,d3219533-d5f4-4b01-9281-834f6cb0019b,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0


##### Data Test

In [ ]:
clickstream_test_w2 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-10-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2021-12-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w2.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
7666792,10470a3a-3c44-4b1d-b6cb-ec7f213e58e2,ADD_TO_CART,2021-10-01T00:50:02.490410Z,b1ba877e-946c-4255-91f6-f2b9f56a3447,MOBILE,"{'product_id': 48476, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
7660820,77fc4049-b3e3-4f24-b954-a42bccd06f7a,SCROLL,2021-10-01T00:50:05.285009Z,db74e433-83e8-44f7-b764-af0a2aefa2b8,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
9664279,ac5f8f84-8277-41ba-af9f-bb0bd4ca9288,ADD_PROMO,2021-10-01T00:50:10.503639Z,53787773-2957-43a8-9911-75d0444e38e3,MOBILE,"{'promo_code': 'LIBURDONG', 'promo_amount': 2335}",1,0,0,0,0,0,0,0,0,1,0
12414371,d058ee1c-3638-4acb-999c-b7af52ece762,HOMEPAGE,2021-10-01T00:50:17.111736Z,fe7d2de3-47aa-4b50-aff2-5bcc758fdeb0,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
10169390,675575ca-6b03-4714-a5b6-a2f7153c793f,SCROLL,2021-10-01T00:50:20.008170Z,2a16b318-55e5-4e3f-b515-8ee3599771ca,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620671,1632e63c-b373-403b-b784-990b92119207,BOOKING,2021-12-01T00:49:09.130483Z,a6d518eb-08a6-4b86-a792-6e81e7369ddf,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
5798800,639c9f74-88bf-46f4-b421-d128ac3a1b78,CLICK,2021-12-01T00:49:18.485172Z,25ae31ad-21a1-4073-aeeb-752bf8b0634d,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
4515700,d407ba8c-dbaa-49eb-8aa6-98f22feaa242,BOOKING,2021-12-01T00:49:23.181006Z,660c0fdd-69be-43ea-8057-814ec2322457,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
3002670,121dcef8-8cfc-4b10-bce4-11837d7a7420,SCROLL,2021-12-01T00:49:49.554806Z,a261abdc-6818-4129-b7ca-a93b6214f502,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0


#### Window #3 (1 Aug 21 - 31 Jan 22)

##### Data Train

In [ ]:
clickstream_train_w3 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-08-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2021-12-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w3.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
4381704,172f052d-cf11-4ec0-a02f-5bd175f5df79,SCROLL,2021-08-01T00:50:04.312665Z,ae4c3010-ae78-4e31-b86a-1f672037a9ad,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
6457803,3d769f1e-c83e-4032-a6d8-7c90d13dd075,SCROLL,2021-08-01T00:50:06.366545Z,4e908f67-438d-46ef-8273-da15e3927ecf,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
5637434,788c97b8-7847-4c5e-b4fd-e12e11630899,HOMEPAGE,2021-08-01T00:50:10.106319Z,ebfe9e34-4a5f-4853-9c9e-d867c53fa9a2,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
11082412,dd462fef-5ebf-425e-b8b2-2f2270ebfcc7,HOMEPAGE,2021-08-01T00:50:49.120477Z,a4004b47-dd0b-4f36-b6c4-34e7fc6ef7e9,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
8269915,89c839ce-8274-4ce4-a623-49c99bc2f41b,ADD_TO_CART,2021-08-01T00:51:01.107933Z,916ed690-65df-438a-ac24-41cc93b4ba29,WEB,"{'product_id': 4069, 'quantity': 3, 'item_pric...",0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620671,1632e63c-b373-403b-b784-990b92119207,BOOKING,2021-12-01T00:49:09.130483Z,a6d518eb-08a6-4b86-a792-6e81e7369ddf,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
5798800,639c9f74-88bf-46f4-b421-d128ac3a1b78,CLICK,2021-12-01T00:49:18.485172Z,25ae31ad-21a1-4073-aeeb-752bf8b0634d,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
4515700,d407ba8c-dbaa-49eb-8aa6-98f22feaa242,BOOKING,2021-12-01T00:49:23.181006Z,660c0fdd-69be-43ea-8057-814ec2322457,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
3002670,121dcef8-8cfc-4b10-bce4-11837d7a7420,SCROLL,2021-12-01T00:49:49.554806Z,a261abdc-6818-4129-b7ca-a93b6214f502,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0


##### Data Test

In [ ]:
clickstream_test_w3 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-12-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-02-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w3.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
6997255,e23a8396-63bb-494f-a33f-8ffc9083a418,HOMEPAGE,2021-12-01T00:50:03.701623Z,1cfe8abb-4e15-4278-ad2b-10deea6b158e,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1
9800769,954e8222-7133-4512-a585-5da16f4f8e01,ADD_TO_CART,2021-12-01T00:50:10.193071Z,98651bb3-fb3d-4297-9b43-a12b891ed01a,MOBILE,"{'product_id': 5701, 'quantity': 1, 'item_pric...",0,1,0,0,0,0,0,0,0,1,0
1187928,45b01d55-8bd0-4018-b7a5-de1b802a0fb8,BOOKING,2021-12-01T00:50:19.138497Z,a793fa06-c0fd-4d92-bb23-54a3e798e7db,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
10997886,5884567a-f7dc-49a1-99e4-a29127458c8b,SCROLL,2021-12-01T00:50:34.755735Z,f181cad8-dce9-4168-a8ce-666e5b04c634,WEB,NaN,0,0,0,0,0,0,0,1,0,0,1
3423496,750400c2-118b-4be8-8217-0e0e49fb950f,CLICK,2021-12-01T00:50:36.155070Z,03d72a9c-7951-4376-92f7-d5beefda02b3,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249585,4201bd2d-9da2-4f0d-bdc3-063aaa9f2337,CLICK,2022-02-01T00:49:34.928154Z,a1474f0c-8f91-4143-baca-d6c489057ba1,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
2018153,c39c38c2-219f-4bae-a7c1-806f59a65d33,ITEM_DETAIL,2022-02-01T00:49:35.679442Z,31eb7030-0558-460b-b08f-1d3574ff1bb1,MOBILE,NaN,0,0,0,0,0,1,0,0,0,1,0
10145297,84036f10-868a-4e2e-95ed-a6e772466858,BOOKING,2022-02-01T00:49:39.938814Z,62e273aa-a7ce-466f-ba23-2374c3725824,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
8443896,a0f2bfec-3568-49f3-9f7b-0099dae65c4f,CLICK,2022-02-01T00:49:47.867782Z,34cecde8-2932-40ce-9fe0-f29d238f40cb,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0


#### Windows #4 (1 Okt 21 - 31 Mar 22)

##### Data Train

In [ ]:
clickstream_train_w4 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-10-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-02-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w4.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
7666792,10470a3a-3c44-4b1d-b6cb-ec7f213e58e2,ADD_TO_CART,2021-10-01T00:50:02.490410Z,b1ba877e-946c-4255-91f6-f2b9f56a3447,MOBILE,"{'product_id': 48476, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
7660820,77fc4049-b3e3-4f24-b954-a42bccd06f7a,SCROLL,2021-10-01T00:50:05.285009Z,db74e433-83e8-44f7-b764-af0a2aefa2b8,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
9664279,ac5f8f84-8277-41ba-af9f-bb0bd4ca9288,ADD_PROMO,2021-10-01T00:50:10.503639Z,53787773-2957-43a8-9911-75d0444e38e3,MOBILE,"{'promo_code': 'LIBURDONG', 'promo_amount': 2335}",1,0,0,0,0,0,0,0,0,1,0
12414371,d058ee1c-3638-4acb-999c-b7af52ece762,HOMEPAGE,2021-10-01T00:50:17.111736Z,fe7d2de3-47aa-4b50-aff2-5bcc758fdeb0,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
10169390,675575ca-6b03-4714-a5b6-a2f7153c793f,SCROLL,2021-10-01T00:50:20.008170Z,2a16b318-55e5-4e3f-b515-8ee3599771ca,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249585,4201bd2d-9da2-4f0d-bdc3-063aaa9f2337,CLICK,2022-02-01T00:49:34.928154Z,a1474f0c-8f91-4143-baca-d6c489057ba1,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
2018153,c39c38c2-219f-4bae-a7c1-806f59a65d33,ITEM_DETAIL,2022-02-01T00:49:35.679442Z,31eb7030-0558-460b-b08f-1d3574ff1bb1,MOBILE,NaN,0,0,0,0,0,1,0,0,0,1,0
10145297,84036f10-868a-4e2e-95ed-a6e772466858,BOOKING,2022-02-01T00:49:39.938814Z,62e273aa-a7ce-466f-ba23-2374c3725824,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
8443896,a0f2bfec-3568-49f3-9f7b-0099dae65c4f,CLICK,2022-02-01T00:49:47.867782Z,34cecde8-2932-40ce-9fe0-f29d238f40cb,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0


##### Data Test

In [ ]:
clickstream_test_w4 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2022-02-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-04-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w4.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
1951884,9edd4c44-0d65-4045-b8e3-e634f9ec6cce,CLICK,2022-02-01T00:50:04.204423Z,631a3a54-c06c-455f-b7ff-fc1d029b4cc1,WEB,NaN,0,0,0,1,0,0,0,0,0,0,1
10852225,ab2b6b46-58a7-42d5-9bea-572e41e8e64a,SEARCH,2022-02-01T00:50:13.025329Z,b6f19e42-7ff2-433c-8245-b9fc1c991ee5,MOBILE,{'search_keywords': 'T-Shirt'},0,0,0,0,0,0,0,0,1,1,0
11681241,e3720414-e2c6-4d65-a65f-1abcd3107435,HOMEPAGE,2022-02-01T00:50:15.116855Z,585d02e8-70b8-4751-84da-f5b799b49e6a,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
4746180,37a97a63-b45e-4999-a44b-7b257c4d86ad,SCROLL,2022-02-01T00:50:32.391844Z,d97102a4-ae21-4481-ac4f-781b5c695b2f,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
8212807,e385c673-5114-4077-bd4d-6ff881d6bc00,SEARCH,2022-02-01T00:50:34.706989Z,f720e5c0-7b8b-43ac-a57e-93213dd9050c,MOBILE,{'search_keywords': 'Tas Wanita'},0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830712,b350da3d-f76a-4314-b13f-4470539411de,BOOKING,2022-04-01T00:49:11.861914Z,876b798d-de5f-48cb-a476-b71048606859,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
6137597,93b6c9f7-c751-42f3-a146-7f007ca18286,ADD_TO_CART,2022-04-01T00:49:19.496158Z,af84fca3-b37a-4509-86aa-15e5dbd84a94,MOBILE,"{'product_id': 11210, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
12537221,27a2a173-7316-41ca-aa30-7bcf4f6da922,ADD_TO_CART,2022-04-01T00:49:27.349797Z,d412e717-fac6-46c7-8a93-e8933968446e,MOBILE,"{'product_id': 30624, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
8560890,9ac9bcf1-ce39-4764-9a13-a554dacb64ad,HOMEPAGE,2022-04-01T00:49:38.801455Z,49135f18-3077-42e7-a66a-162d796a2364,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1


#### Window #5 (1 Des 21 - 31 Mei 22)

##### Data Train

In [ ]:
clickstream_train_w5 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2021-12-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-04-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w5.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
6997255,e23a8396-63bb-494f-a33f-8ffc9083a418,HOMEPAGE,2021-12-01T00:50:03.701623Z,1cfe8abb-4e15-4278-ad2b-10deea6b158e,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1
9800769,954e8222-7133-4512-a585-5da16f4f8e01,ADD_TO_CART,2021-12-01T00:50:10.193071Z,98651bb3-fb3d-4297-9b43-a12b891ed01a,MOBILE,"{'product_id': 5701, 'quantity': 1, 'item_pric...",0,1,0,0,0,0,0,0,0,1,0
1187928,45b01d55-8bd0-4018-b7a5-de1b802a0fb8,BOOKING,2021-12-01T00:50:19.138497Z,a793fa06-c0fd-4d92-bb23-54a3e798e7db,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
10997886,5884567a-f7dc-49a1-99e4-a29127458c8b,SCROLL,2021-12-01T00:50:34.755735Z,f181cad8-dce9-4168-a8ce-666e5b04c634,WEB,NaN,0,0,0,0,0,0,0,1,0,0,1
3423496,750400c2-118b-4be8-8217-0e0e49fb950f,CLICK,2021-12-01T00:50:36.155070Z,03d72a9c-7951-4376-92f7-d5beefda02b3,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830712,b350da3d-f76a-4314-b13f-4470539411de,BOOKING,2022-04-01T00:49:11.861914Z,876b798d-de5f-48cb-a476-b71048606859,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
6137597,93b6c9f7-c751-42f3-a146-7f007ca18286,ADD_TO_CART,2022-04-01T00:49:19.496158Z,af84fca3-b37a-4509-86aa-15e5dbd84a94,MOBILE,"{'product_id': 11210, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
12537221,27a2a173-7316-41ca-aa30-7bcf4f6da922,ADD_TO_CART,2022-04-01T00:49:27.349797Z,d412e717-fac6-46c7-8a93-e8933968446e,MOBILE,"{'product_id': 30624, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
8560890,9ac9bcf1-ce39-4764-9a13-a554dacb64ad,HOMEPAGE,2022-04-01T00:49:38.801455Z,49135f18-3077-42e7-a66a-162d796a2364,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1


##### Data Test

In [ ]:
clickstream_test_w5 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2022-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-06-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w5.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
8707607,30ba3d76-4e84-47bc-ae35-fda23062e37d,CLICK,2022-04-01T00:50:05.814624Z,3ae58c23-4a62-4426-a1f1-4fdbde8ece52,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
1964755,c545e59f-a37d-4546-86ce-cc06075235b5,HOMEPAGE,2022-04-01T00:50:09.590991Z,4aad7b12-01cd-46aa-8329-e13d0a012229,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
8995285,865f8623-542d-407d-ae03-4a6656f56fb1,BOOKING,2022-04-01T00:50:17.236731Z,d91a69b0-c246-4335-9373-dc1a1beb3c27,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
11308267,ab53eca8-e226-4b6c-8014-66f645c71e53,SEARCH,2022-04-01T00:50:22.733693Z,55ce48db-5e58-4955-9af0-5e8f1ce8dbbf,MOBILE,{'search_keywords': 'Dress Kondangan'},0,0,0,0,0,0,0,0,1,1,0
4252266,cf8abdf7-9c61-48b3-afb8-2b3b5da5b4aa,CLICK,2022-04-01T00:50:22.981649Z,01e4f22f-daa5-4476-912a-11637dccd464,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142099,bf9e2519-c3d1-41ae-ab63-527d2f90fd25,CLICK,2022-06-01T00:49:41.232873Z,0ab43f99-740a-4da4-837d-d1289381e0a6,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
2034948,9f27dd63-9db7-4251-9590-2dc80f1ce023,CLICK,2022-06-01T00:49:44.306015Z,30389c2d-b6f5-4455-be2b-f22a7b1b0451,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
684765,fa9c1693-f9ba-4922-bc85-49638948aaa4,CLICK,2022-06-01T00:49:56.132817Z,a1bfe6d8-c826-45a6-9fbb-c6772394ef08,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
4127105,a19c3f25-91da-4f97-8c88-13773e64b733,HOMEPAGE,2022-06-01T00:49:57.911893Z,fd74f1f7-3035-45a7-b0e6-0f88a7d46969,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0


#### Window #6 (1 Feb 22 - 1 Aug 22)

##### Data Train

In [ ]:
clickstream_train_w6 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2022-02-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-06-01T00:49:59.408424Z'))]

In [ ]:
clickstream_train_w6.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
1951884,9edd4c44-0d65-4045-b8e3-e634f9ec6cce,CLICK,2022-02-01T00:50:04.204423Z,631a3a54-c06c-455f-b7ff-fc1d029b4cc1,WEB,NaN,0,0,0,1,0,0,0,0,0,0,1
10852225,ab2b6b46-58a7-42d5-9bea-572e41e8e64a,SEARCH,2022-02-01T00:50:13.025329Z,b6f19e42-7ff2-433c-8245-b9fc1c991ee5,MOBILE,{'search_keywords': 'T-Shirt'},0,0,0,0,0,0,0,0,1,1,0
11681241,e3720414-e2c6-4d65-a65f-1abcd3107435,HOMEPAGE,2022-02-01T00:50:15.116855Z,585d02e8-70b8-4751-84da-f5b799b49e6a,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
4746180,37a97a63-b45e-4999-a44b-7b257c4d86ad,SCROLL,2022-02-01T00:50:32.391844Z,d97102a4-ae21-4481-ac4f-781b5c695b2f,MOBILE,NaN,0,0,0,0,0,0,0,1,0,1,0
8212807,e385c673-5114-4077-bd4d-6ff881d6bc00,SEARCH,2022-02-01T00:50:34.706989Z,f720e5c0-7b8b-43ac-a57e-93213dd9050c,MOBILE,{'search_keywords': 'Tas Wanita'},0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142099,bf9e2519-c3d1-41ae-ab63-527d2f90fd25,CLICK,2022-06-01T00:49:41.232873Z,0ab43f99-740a-4da4-837d-d1289381e0a6,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
2034948,9f27dd63-9db7-4251-9590-2dc80f1ce023,CLICK,2022-06-01T00:49:44.306015Z,30389c2d-b6f5-4455-be2b-f22a7b1b0451,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
684765,fa9c1693-f9ba-4922-bc85-49638948aaa4,CLICK,2022-06-01T00:49:56.132817Z,a1bfe6d8-c826-45a6-9fbb-c6772394ef08,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
4127105,a19c3f25-91da-4f97-8c88-13773e64b733,HOMEPAGE,2022-06-01T00:49:57.911893Z,fd74f1f7-3035-45a7-b0e6-0f88a7d46969,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0


##### Data Test

In [ ]:
clickstream_test_w6 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2022-06-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-08-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w6.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
10422558,71f8ce16-2132-4097-8d2e-3971781d111b,CLICK,2022-06-01T00:50:02.079398Z,6ca7d2e9-f980-4d31-ad21-dd2b2842b298,WEB,NaN,0,0,0,1,0,0,0,0,0,0,1
4002654,37ac2465-65fa-457d-921d-b189e67a8b68,HOMEPAGE,2022-06-01T00:50:06.578199Z,0d933ec5-d07a-4604-b036-5f35efc0e576,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
12266362,6f975e63-80cc-496a-82d5-463861ae8477,PROMO_PAGE,2022-06-01T00:50:17.141245Z,bfe911eb-538f-46e7-a975-e423c339b340,MOBILE,NaN,0,0,0,0,0,0,1,0,0,1,0
11221495,d270284c-7764-4137-b838-271a835d3e59,ADD_TO_CART,2022-06-01T00:50:24.144794Z,8a3b1a96-0152-4cb6-8a34-3718c4e7adc6,MOBILE,"{'product_id': 33697, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
6258831,4356b7ae-388a-4aff-9dfb-1c755d6733b3,ADD_TO_CART,2022-06-01T00:50:25.232951Z,6f9e2a45-9827-488e-b478-e1c225727551,MOBILE,"{'product_id': 44201, 'quantity': 1, 'item_pri...",0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11980458,95168514-c4a8-4611-8739-c45dc6374b43,HOMEPAGE,2022-08-01T00:23:04.821469Z,9eaae58f-1ca7-4db3-ac0f-76a3e4497d34,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1
4959631,87a77ed6-e487-45d2-b73b-aa9277a2d717,HOMEPAGE,2022-08-01T00:24:28.200546Z,439df875-64eb-4f13-8871-a57af159bc48,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
12803757,f837b3a2-7908-42df-8969-3f879ebb4c25,HOMEPAGE,2022-08-01T00:29:20.408424Z,5b27329b-c28e-42a6-9c92-df4a3ef0fe2e,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
10061205,48bdb39a-195d-4acf-835a-35556390f7dd,CLICK,2022-08-01T00:31:53.957678Z,a0d5d325-966f-44f0-99ed-b8670ef4601f,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0


#### Window #7 (1 Apr 22 - 1 Aug 22)

##### Data Test

In [ ]:
clickstream_test_w7 = clickstream[(pd.to_datetime(clickstream['event_time']) >= pd.to_datetime('2022-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(clickstream['event_time']) < pd.to_datetime('2022-08-01T00:49:59.408424Z'))]

In [ ]:
clickstream_test_w7.sort_values(by='event_time')

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_name_ADD_PROMO,event_name_ADD_TO_CART,event_name_BOOKING,event_name_CLICK,event_name_HOMEPAGE,event_name_ITEM_DETAIL,event_name_PROMO_PAGE,event_name_SCROLL,event_name_SEARCH,traffic_source_MOBILE,traffic_source_WEB
8707607,30ba3d76-4e84-47bc-ae35-fda23062e37d,CLICK,2022-04-01T00:50:05.814624Z,3ae58c23-4a62-4426-a1f1-4fdbde8ece52,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
1964755,c545e59f-a37d-4546-86ce-cc06075235b5,HOMEPAGE,2022-04-01T00:50:09.590991Z,4aad7b12-01cd-46aa-8329-e13d0a012229,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
8995285,865f8623-542d-407d-ae03-4a6656f56fb1,BOOKING,2022-04-01T00:50:17.236731Z,d91a69b0-c246-4335-9373-dc1a1beb3c27,MOBILE,{'payment_status': 'Success'},0,0,1,0,0,0,0,0,0,1,0
11308267,ab53eca8-e226-4b6c-8014-66f645c71e53,SEARCH,2022-04-01T00:50:22.733693Z,55ce48db-5e58-4955-9af0-5e8f1ce8dbbf,MOBILE,{'search_keywords': 'Dress Kondangan'},0,0,0,0,0,0,0,0,1,1,0
4252266,cf8abdf7-9c61-48b3-afb8-2b3b5da5b4aa,CLICK,2022-04-01T00:50:22.981649Z,01e4f22f-daa5-4476-912a-11637dccd464,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11980458,95168514-c4a8-4611-8739-c45dc6374b43,HOMEPAGE,2022-08-01T00:23:04.821469Z,9eaae58f-1ca7-4db3-ac0f-76a3e4497d34,WEB,NaN,0,0,0,0,1,0,0,0,0,0,1
4959631,87a77ed6-e487-45d2-b73b-aa9277a2d717,HOMEPAGE,2022-08-01T00:24:28.200546Z,439df875-64eb-4f13-8871-a57af159bc48,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
12803757,f837b3a2-7908-42df-8969-3f879ebb4c25,HOMEPAGE,2022-08-01T00:29:20.408424Z,5b27329b-c28e-42a6-9c92-df4a3ef0fe2e,MOBILE,NaN,0,0,0,0,1,0,0,0,0,1,0
10061205,48bdb39a-195d-4acf-835a-35556390f7dd,CLICK,2022-08-01T00:31:53.957678Z,a0d5d325-966f-44f0-99ed-b8670ef4601f,MOBILE,NaN,0,0,0,1,0,0,0,0,0,1,0


### Features Engineering

#### Window #1 (1 Apr 21 - 31 Sep 21)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w1_new=clickstream_train_w1.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w1_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00016815-cf29-45fa-8974-17c6796aab29,23,0,1,1,7,2.0,3,1,5,3,2021-04-29T02:15:32.180746Z,2021-04-30T15:55:09.180746Z,1,0
1,0002f9d7-4d84-4584-89d7-ec258640b6ff,32,1,1,1,7,4.0,0,3,10,5,2021-05-16T15:37:35.288598Z,2021-05-17T00:50:24.288598Z,1,0
2,00051df2-4589-43ec-9ef7-d5527ad793a5,18,0,5,0,4,2.0,3,0,3,1,2021-07-04T23:30:23.236904Z,2021-07-05T00:03:53.236904Z,1,0
3,000547cd-a3f9-4f2f-a930-adfa905cf7de,1,0,0,0,0,1.0,0,0,0,0,2021-04-03T18:42:59.316417Z,2021-04-03T18:42:59.316417Z,1,0
4,0005a755-1028-47c3-a182-b3fa533bff9f,5,1,1,0,1,1.0,0,0,1,0,2021-05-18T05:36:03.769317Z,2021-06-23T23:24:43.769317Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91618,fffb5087-c192-4fec-8665-2da210104dd3,10,0,1,1,3,1.0,2,0,1,1,2021-04-09T14:53:39.603497Z,2021-04-11T10:50:44.603497Z,1,0
91619,fffbc161-54c3-4da9-9868-df9fb67e52a4,8,0,1,1,0,3.0,1,0,1,1,2021-05-24T02:26:10.150847Z,2021-05-24T13:12:12.150847Z,1,0
91620,fffc22b9-39aa-4ce6-a4ca-14fd84fdb0cd,36,1,1,1,13,7.0,3,2,4,4,2021-06-26T11:14:00.806794Z,2021-07-09T04:03:37.806794Z,1,0
91621,fffc24d3-6e47-4df3-9d6c-3b89c63fb5fc,50,0,10,1,13,4.0,10,1,8,3,2021-05-28T21:21:45.818614Z,2021-07-29T19:57:38.818614Z,1,0


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w1_new=clickstream_test_w1.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w1_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000f2fc-1875-4c88-9398-5b7386b14ca6,13,1,1,1,1,2,2,1,2,2,2021-08-01T01:22:52.970951Z,2021-08-01T06:31:03.970951Z,1,0
1,0004a700-7db4-4127-a592-02ac7d7a618b,16,0,3,1,3,3,0,1,4,1,2021-08-29T18:17:39.310591Z,2021-08-29T18:27:09.241025Z,1,0
2,0004d5bf-7b3a-42b0-9c69-c5454b0b84de,3,0,1,0,0,1,0,0,0,1,2021-09-26T21:42:01.630204Z,2021-09-29T14:20:50.630204Z,1,0
3,00054db3-5873-44a1-895e-c6ac003811cd,5,0,1,1,0,1,2,0,0,0,2021-09-12T21:04:45.623733Z,2021-09-12T21:45:36.492621Z,0,1
4,0005a755-1028-47c3-a182-b3fa533bff9f,1,0,0,1,0,0,0,0,0,0,2021-08-17T23:35:59.769317Z,2021-08-17T23:35:59.769317Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56269,fff50c56-b3a5-46e3-944f-c1018b7748e2,6,0,1,1,1,1,1,0,0,1,2021-09-11T17:42:39.583807Z,2021-09-12T02:04:02.583807Z,1,0
56270,fff67eff-36b8-41c4-89fd-4c44d2f43724,30,0,1,1,8,8,8,1,1,2,2021-08-23T04:39:26.291935Z,2021-08-30T23:19:12.291935Z,1,0
56271,fff6c85c-cd43-4ffd-8da2-249cb11b969d,8,1,1,1,2,1,1,0,1,0,2021-08-03T00:50:43.521036Z,2021-09-29T21:55:57.521036Z,1,0
56272,fff6e653-4417-492b-889a-f4afea12a8fd,12,1,3,0,3,1,2,1,0,1,2021-09-27T02:00:43.368694Z,2021-09-30T22:19:58.368694Z,1,0


#### Window #2 (1 Jun 21 - 31 Nov 21)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w2_new=clickstream_train_w2.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w2_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000f2fc-1875-4c88-9398-5b7386b14ca6,13,1,1,1,1,2.0,2,1,2,2,2021-08-01T01:22:52.970951Z,2021-08-01T06:31:03.970951Z,1,0
1,0004a700-7db4-4127-a592-02ac7d7a618b,16,0,3,1,3,3.0,0,1,4,1,2021-08-29T18:17:39.310591Z,2021-08-29T18:27:09.241025Z,1,0
2,0004d5bf-7b3a-42b0-9c69-c5454b0b84de,3,0,1,0,0,1.0,0,0,0,1,2021-09-26T21:42:01.630204Z,2021-09-29T14:20:50.630204Z,1,0
3,00051df2-4589-43ec-9ef7-d5527ad793a5,18,0,5,0,4,2.0,3,0,3,1,2021-07-04T23:30:23.236904Z,2021-07-05T00:03:53.236904Z,1,0
4,00054db3-5873-44a1-895e-c6ac003811cd,5,0,1,1,0,1.0,2,0,0,0,2021-09-12T21:04:45.623733Z,2021-09-12T21:45:36.492621Z,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99545,fffaa5ce-6f02-4576-921a-f13576c73070,6,0,2,1,0,1.0,1,0,1,0,2021-09-01T08:11:16.903358Z,2021-09-02T21:29:19.903358Z,1,0
99546,fffad5a5-af92-4eb5-a6fa-dccf3d35b621,12,1,1,1,3,1.0,1,1,2,1,2021-07-10T12:56:29.994991Z,2021-07-19T19:23:42.994991Z,1,0
99547,fffc22b9-39aa-4ce6-a4ca-14fd84fdb0cd,36,1,1,1,13,7.0,3,2,4,4,2021-06-26T11:14:00.806794Z,2021-07-09T04:03:37.806794Z,1,0
99548,fffc24d3-6e47-4df3-9d6c-3b89c63fb5fc,45,0,8,1,12,3.0,9,1,8,3,2021-06-01T16:26:02.818614Z,2021-07-29T19:57:38.818614Z,1,0


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w2_new=clickstream_test_w2.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w2_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000da55-698f-4d22-8b44-3ca192f2f961,28,0,1,1,5,5,7,2,4,3,2021-11-08T13:48:16.917948Z,2021-11-15T16:22:22.917948Z,1,0
1,000130b9-07d1-4374-99b8-89d769c46c9f,47,0,1,1,11,12,6,1,11,4,2021-10-20T20:50:34.511904Z,2021-10-21T21:09:24.511904Z,1,0
2,00016d51-8e61-4404-afca-ada83a4b8787,4,0,1,0,1,1,0,0,0,1,2021-10-05T17:44:38.721623Z,2021-11-06T12:26:04.721623Z,1,0
3,000288dc-e7cf-44b9-9f2d-270015fd83c6,65,0,1,1,13,12,10,4,11,13,2021-11-20T00:19:45.288196Z,2021-11-20T19:46:37.288196Z,1,0
4,00041c26-ab42-41ca-b005-e56891af5bfe,11,1,4,0,2,1,1,1,0,1,2021-11-04T07:41:28.314034Z,2021-11-28T14:06:13.314034Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59785,fffca6ee-1822-40f7-a9fd-88362e464378,19,1,1,1,7,1,1,0,2,5,2021-10-07T18:27:23.627533Z,2021-10-09T03:08:03.627533Z,1,0
59786,fffdd2eb-33e4-4566-9846-e788c96c4109,33,1,1,1,9,6,2,1,8,4,2021-11-27T00:47:04.520646Z,2021-11-28T08:06:14.520646Z,1,0
59787,fffe8737-cfd5-41c9-ad94-593b6a593839,6,0,1,1,0,1,2,0,1,0,2021-10-14T11:15:36.276146Z,2021-10-20T20:53:33.276146Z,1,0
59788,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,19,1,2,1,3,5,0,3,2,2,2021-10-04T21:17:12.231938Z,2021-10-08T15:22:31.231938Z,0,1


#### Window #3 (1 Aug 21 - 31 Jan 22)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w3_new=clickstream_train_w3.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w3_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000da55-698f-4d22-8b44-3ca192f2f961,28,0,1,1,5,5,7,2,4,3,2021-11-08T13:48:16.917948Z,2021-11-15T16:22:22.917948Z,1,0
1,0000f2fc-1875-4c88-9398-5b7386b14ca6,13,1,1,1,1,2,2,1,2,2,2021-08-01T01:22:52.970951Z,2021-08-01T06:31:03.970951Z,1,0
2,000130b9-07d1-4374-99b8-89d769c46c9f,47,0,1,1,11,12,6,1,11,4,2021-10-20T20:50:34.511904Z,2021-10-21T21:09:24.511904Z,1,0
3,00016d51-8e61-4404-afca-ada83a4b8787,4,0,1,0,1,1,0,0,0,1,2021-10-05T17:44:38.721623Z,2021-11-06T12:26:04.721623Z,1,0
4,000288dc-e7cf-44b9-9f2d-270015fd83c6,65,0,1,1,13,12,10,4,11,13,2021-11-20T00:19:45.288196Z,2021-11-20T19:46:37.288196Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107797,fffca6ee-1822-40f7-a9fd-88362e464378,19,1,1,1,7,1,1,0,2,5,2021-10-07T18:27:23.627533Z,2021-10-09T03:08:03.627533Z,1,0
107798,fffdd2eb-33e4-4566-9846-e788c96c4109,33,1,1,1,9,6,2,1,8,4,2021-11-27T00:47:04.520646Z,2021-11-28T08:06:14.520646Z,1,0
107799,fffe8737-cfd5-41c9-ad94-593b6a593839,6,0,1,1,0,1,2,0,1,0,2021-10-14T11:15:36.276146Z,2021-10-20T20:53:33.276146Z,1,0
107800,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,19,1,2,1,3,5,0,3,2,2,2021-10-04T21:17:12.231938Z,2021-10-08T15:22:31.231938Z,0,1


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w3_new=clickstream_test_w3.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w3_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00016d51-8e61-4404-afca-ada83a4b8787,1,0,0,1,0,0.0,0,0,0,0,2021-12-08T07:34:00.721623Z,2021-12-08T07:34:00.721623Z,1,0
1,00041c26-ab42-41ca-b005-e56891af5bfe,1,0,0,1,0,0.0,0,0,0,0,2021-12-07T17:37:51.314034Z,2021-12-07T17:37:51.314034Z,1,0
2,0004216f-69fa-4c8b-8a6d-a0534f41e436,6,1,1,1,1,1.0,0,0,0,1,2021-12-28T23:36:41.372757Z,2021-12-29T17:27:45.372757Z,1,0
3,000446a5-b481-4512-bd0a-cd5eb92b81c0,11,1,3,1,2,1.0,0,0,3,0,2022-01-18T21:09:25.510903Z,2022-01-25T20:38:56.510903Z,1,0
4,00082f92-6a6b-4644-839b-65d6bcf6a3be,6,0,2,1,0,1.0,0,1,1,0,2022-01-19T13:52:04.603265Z,2022-01-19T18:03:18.603265Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62466,fffd5e21-ff89-4c97-8fe5-5d59520d0699,25,0,1,1,10,5.0,1,0,4,3,2021-12-08T13:57:42.541124Z,2022-01-05T21:57:55.541124Z,1,0
62467,fffda767-c504-41f7-8371-408cf7b8ca9a,8,0,1,1,4,1.0,1,0,0,0,2021-12-23T03:00:03.294325Z,2021-12-29T22:46:57.294325Z,1,0
62468,fffdfec6-ecb2-4f49-bef8-34e4995d099e,6,0,1,1,1,1.0,0,0,1,1,2021-12-02T21:09:20.243566Z,2021-12-06T04:25:32.243566Z,1,0
62469,fffe2531-f1a9-4afc-b5cd-fc6a5465493b,5,0,1,0,1,1.0,0,0,1,1,2022-01-20T07:12:16.425689Z,2022-01-30T20:50:22.425689Z,1,0


#### Windows #4 (1 Okt 21 - 31 Mar 22)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w4_new=clickstream_train_w4.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w4_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000da55-698f-4d22-8b44-3ca192f2f961,28,0,1,1,5,5.0,7,2,4,3,2021-11-08T13:48:16.917948Z,2021-11-15T16:22:22.917948Z,1,0
1,000130b9-07d1-4374-99b8-89d769c46c9f,47,0,1,1,11,12.0,6,1,11,4,2021-10-20T20:50:34.511904Z,2021-10-21T21:09:24.511904Z,1,0
2,00016d51-8e61-4404-afca-ada83a4b8787,5,0,1,1,1,1.0,0,0,0,1,2021-10-05T17:44:38.721623Z,2021-12-08T07:34:00.721623Z,1,0
3,000288dc-e7cf-44b9-9f2d-270015fd83c6,65,0,1,1,13,12.0,10,4,11,13,2021-11-20T00:19:45.288196Z,2021-11-20T19:46:37.288196Z,1,0
4,00041c26-ab42-41ca-b005-e56891af5bfe,12,1,4,1,2,1.0,1,1,0,1,2021-11-04T07:41:28.314034Z,2021-12-07T17:37:51.314034Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113933,fffe2531-f1a9-4afc-b5cd-fc6a5465493b,5,0,1,0,1,1.0,0,0,1,1,2022-01-20T07:12:16.425689Z,2022-01-30T20:50:22.425689Z,1,0
113934,fffe53ca-288b-4bcb-825a-c54f6169870e,7,0,4,1,0,2.0,0,0,0,0,2022-01-13T01:31:29.511581Z,2022-01-15T15:37:49.511581Z,1,0
113935,fffe8737-cfd5-41c9-ad94-593b6a593839,6,0,1,1,0,1.0,2,0,1,0,2021-10-14T11:15:36.276146Z,2021-10-20T20:53:33.276146Z,1,0
113936,ffffce53-1ba7-4d7d-a14a-ffa7079cfbff,19,1,2,1,3,5.0,0,3,2,2,2021-10-04T21:17:12.231938Z,2021-10-08T15:22:31.231938Z,0,1


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w4_new=clickstream_test_w4.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w4_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00005746-17ec-449c-9c5c-699a3ea9304a,66,0,1,1,23,9,8,1,12,11,2022-03-08T23:17:19.510057Z,2022-03-08T23:21:35.510057Z,1,0
1,0000a729-3de3-4613-9022-d12b7fa06e75,1,0,0,0,0,1,0,0,0,0,2022-03-30T17:27:21.273818Z,2022-03-30T17:27:21.273818Z,1,0
2,0001c892-ebc8-42ac-bddf-ce4dcd4b4658,53,0,1,1,15,22,6,1,6,1,2022-03-13T21:55:56.555697Z,2022-03-25T03:07:27.555697Z,1,0
3,0002451c-6217-4d05-8cd6-e864bddc4287,4,0,1,1,0,1,1,0,0,0,2022-03-11T20:12:23.759800Z,2022-03-29T21:16:08.759800Z,1,0
4,00052824-fb09-42db-8796-57bf4bb2b995,29,1,6,1,8,4,3,2,3,1,2022-03-07T04:34:11.130066Z,2022-03-07T04:50:12.537929Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63505,fffa8c63-3872-40b8-9ead-a01713334e0b,1,0,0,1,0,0,0,0,0,0,2022-02-09T02:58:35.903794Z,2022-02-09T02:58:35.903794Z,1,0
63506,fffd5ce8-43e0-4747-a028-a6e67ea44c53,3,0,1,1,0,0,1,0,0,0,2022-02-01T08:04:04.574889Z,2022-02-04T11:29:09.574889Z,1,0
63507,fffdbe04-e042-4859-9336-7cf4eda9a1f3,4,0,1,1,1,1,0,0,0,0,2022-03-26T16:36:03.669060Z,2022-03-30T03:24:46.669060Z,1,0
63508,fffe2531-f1a9-4afc-b5cd-fc6a5465493b,4,0,0,1,0,2,0,0,0,1,2022-02-03T09:08:38.425689Z,2022-02-17T11:00:11.425689Z,1,0


#### Window #5 (1 Des 21 - 31 Mei 22)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w5_new=clickstream_train_w5.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w5_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00005746-17ec-449c-9c5c-699a3ea9304a,66,0,1,1,23,9.0,8,1,12,11,2022-03-08T23:17:19.510057Z,2022-03-08T23:21:35.510057Z,1,0
1,0000a729-3de3-4613-9022-d12b7fa06e75,1,0,0,0,0,1.0,0,0,0,0,2022-03-30T17:27:21.273818Z,2022-03-30T17:27:21.273818Z,1,0
2,00016d51-8e61-4404-afca-ada83a4b8787,1,0,0,1,0,0.0,0,0,0,0,2021-12-08T07:34:00.721623Z,2021-12-08T07:34:00.721623Z,1,0
3,0001c892-ebc8-42ac-bddf-ce4dcd4b4658,53,0,1,1,15,22.0,6,1,6,1,2022-03-13T21:55:56.555697Z,2022-03-25T03:07:27.555697Z,1,0
4,0002451c-6217-4d05-8cd6-e864bddc4287,4,0,1,1,0,1.0,1,0,0,0,2022-03-11T20:12:23.759800Z,2022-03-29T21:16:08.759800Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117868,fffdbe04-e042-4859-9336-7cf4eda9a1f3,4,0,1,1,1,1.0,0,0,0,0,2022-03-26T16:36:03.669060Z,2022-03-30T03:24:46.669060Z,1,0
117869,fffdfec6-ecb2-4f49-bef8-34e4995d099e,6,0,1,1,1,1.0,0,0,1,1,2021-12-02T21:09:20.243566Z,2021-12-06T04:25:32.243566Z,1,0
117870,fffe2531-f1a9-4afc-b5cd-fc6a5465493b,9,0,1,1,1,3.0,0,0,1,2,2022-01-20T07:12:16.425689Z,2022-02-17T11:00:11.425689Z,1,0
117871,fffe53ca-288b-4bcb-825a-c54f6169870e,7,0,4,1,0,2.0,0,0,0,0,2022-01-13T01:31:29.511581Z,2022-01-15T15:37:49.511581Z,1,0


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w5_new=clickstream_test_w5.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w5_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,0000a729-3de3-4613-9022-d12b7fa06e75,4,0,2,1,1,0.0,0,0,0,0,2022-04-02T01:24:07.273818Z,2022-04-09T02:02:54.273818Z,1,0
1,0002ad9d-3c59-4c9c-b6dc-ae6a561ef718,4,0,1,1,0,1.0,0,0,1,0,2022-04-29T17:01:59.008860Z,2022-04-30T07:06:55.008860Z,0,1
2,0003ff81-e79d-4d8a-8dde-bd76727f6a81,20,0,1,1,5,2.0,2,2,7,0,2022-05-09T21:25:04.667324Z,2022-05-13T19:42:33.667324Z,1,0
3,0005524b-23b1-4406-b7c7-221060c7235f,4,0,1,1,0,1.0,1,0,0,0,2022-04-20T12:54:31.493642Z,2022-04-23T11:12:15.493642Z,1,0
4,0005d22d-d589-43cc-aef8-f9324156236c,23,0,1,1,6,4.0,2,0,7,2,2022-05-18T22:27:47.551962Z,2022-05-19T21:39:33.551962Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66803,fffb3bcd-30ab-4721-a564-daf90769fa8c,21,0,1,1,6,2.0,4,0,1,6,2022-04-21T09:31:05.255284Z,2022-04-28T10:35:31.255284Z,1,0
66804,fffc7d34-066a-4458-83fb-bdf67cc983af,12,0,2,1,4,2.0,0,1,2,0,2022-04-09T17:09:37.714939Z,2022-05-05T17:25:24.714939Z,1,0
66805,fffe0edc-392a-49c1-911c-451c14cb0298,10,1,1,1,1,4.0,0,0,1,1,2022-04-21T21:17:39.711463Z,2022-04-24T10:29:06.711463Z,1,0
66806,fffe23af-9ec7-43cb-9f86-550bb58466bc,12,1,1,1,4,3.0,1,1,0,0,2022-05-06T20:31:05.342560Z,2022-05-11T07:23:55.342560Z,1,0


#### Window #6 (1 Feb 22 - 1 Aug 22)

##### Data Train

In [ ]:
# Group by session_id & Aggregating (train)
clickstream_train_w6_new=clickstream_train_w6.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_train_w6_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00005746-17ec-449c-9c5c-699a3ea9304a,66,0,1,1,23,9.0,8,1,12,11,2022-03-08T23:17:19.510057Z,2022-03-08T23:21:35.510057Z,1,0
1,0000a729-3de3-4613-9022-d12b7fa06e75,5,0,2,1,1,1.0,0,0,0,0,2022-03-30T17:27:21.273818Z,2022-04-09T02:02:54.273818Z,1,0
2,0001c892-ebc8-42ac-bddf-ce4dcd4b4658,53,0,1,1,15,22.0,6,1,6,1,2022-03-13T21:55:56.555697Z,2022-03-25T03:07:27.555697Z,1,0
3,0002451c-6217-4d05-8cd6-e864bddc4287,4,0,1,1,0,1.0,1,0,0,0,2022-03-11T20:12:23.759800Z,2022-03-29T21:16:08.759800Z,1,0
4,0002ad9d-3c59-4c9c-b6dc-ae6a561ef718,4,0,1,1,0,1.0,0,0,1,0,2022-04-29T17:01:59.008860Z,2022-04-30T07:06:55.008860Z,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122595,fffe0edc-392a-49c1-911c-451c14cb0298,10,1,1,1,1,4.0,0,0,1,1,2022-04-21T21:17:39.711463Z,2022-04-24T10:29:06.711463Z,1,0
122596,fffe23af-9ec7-43cb-9f86-550bb58466bc,12,1,1,1,4,3.0,1,1,0,0,2022-05-06T20:31:05.342560Z,2022-05-11T07:23:55.342560Z,1,0
122597,fffe2531-f1a9-4afc-b5cd-fc6a5465493b,4,0,0,1,0,2.0,0,0,0,1,2022-02-03T09:08:38.425689Z,2022-02-17T11:00:11.425689Z,1,0
122598,fffe5ead-3b3d-442b-9b1d-99c91412e8f5,9,1,3,0,0,1.0,0,1,1,2,2022-05-18T15:42:44.951998Z,2022-05-31T08:31:46.951998Z,1,0


##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w6_new=clickstream_test_w6.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w6_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00007415-e0e7-4a33-95e8-5bc749016876,54,0,1,1,18,5,8,1,11,9,2022-07-24T13:12:21.038087Z,2022-07-24T13:16:40.038087Z,0,1
1,0001afa6-3f2c-4cb9-a118-360831dcb892,11,0,2,1,1,4,0,0,1,2,2022-06-18T03:51:48.610015Z,2022-06-28T16:34:01.610015Z,0,1
2,0004474f-f874-4239-a574-0a03ae386a77,5,0,1,1,1,2,0,0,0,0,2022-06-08T02:25:22.387327Z,2022-06-10T22:50:02.387327Z,1,0
3,00057675-97dc-4f4d-8cc2-04e2ce17b1a1,13,0,1,1,3,4,0,1,1,2,2022-06-08T17:10:52.176505Z,2022-06-10T05:48:07.176505Z,1,0
4,00064f7a-6c82-4322-a21f-ed16de1e62cc,16,0,1,1,3,1,4,1,3,2,2022-07-19T22:39:10.187523Z,2022-07-20T14:31:16.187523Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58027,fffcfab3-9995-46cc-98fe-0e142592ae5b,9,0,4,1,0,1,0,0,3,0,2022-07-25T11:52:18.560991Z,2022-07-26T03:12:12.560991Z,1,0
58028,fffe5ead-3b3d-442b-9b1d-99c91412e8f5,1,0,0,1,0,0,0,0,0,0,2022-06-06T16:36:53.951998Z,2022-06-06T16:36:53.951998Z,1,0
58029,fffea676-ae7b-48f0-9266-db56eaf9755c,9,1,1,1,2,1,0,1,1,1,2022-07-18T04:28:16.204042Z,2022-07-18T09:29:33.204042Z,0,1
58030,fffedbf4-5929-4801-a684-5e50ab05c999,10,1,1,1,3,1,0,0,3,0,2022-07-08T15:21:54.697105Z,2022-07-09T00:01:42.697105Z,1,0


#### Window #7 (1 Apr 22 - 1 Aug 22)

##### Data Test

In [ ]:
# Group by session_id & Aggregating (test)
clickstream_test_w7_new=clickstream_test_w7.groupby('session_id').agg(
    event_total=pd.NamedAgg(column='event_id', aggfunc='count'),
    event_add_promo=pd.NamedAgg(column='event_name_ADD_PROMO', aggfunc='sum'),
    event_add_to_cart=pd.NamedAgg(column='event_name_ADD_TO_CART', aggfunc='sum'),
    event_booking=pd.NamedAgg(column='event_name_BOOKING', aggfunc='sum'),
    event_click=pd.NamedAgg(column='event_name_CLICK', aggfunc='sum'),
    event_homepage=pd.NamedAgg(column='event_name_HOMEPAGE', aggfunc='sum'),
    event_item_detail=pd.NamedAgg(column='event_name_ITEM_DETAIL', aggfunc='sum'),
    event_promo_page=pd.NamedAgg(column='event_name_PROMO_PAGE', aggfunc='sum'),
    event_scroll=pd.NamedAgg(column='event_name_SCROLL', aggfunc='sum'),
    event_search=pd.NamedAgg(column='event_name_SEARCH', aggfunc='sum'),
    event_time_start=pd.NamedAgg(column='event_time', aggfunc='min'),
    event_time_end=pd.NamedAgg(column='event_time', aggfunc='max'),
    is_using_mobile=pd.NamedAgg(column='traffic_source_MOBILE', aggfunc='max'),
    is_using_web=pd.NamedAgg(column='traffic_source_WEB', aggfunc='max')
    ).reset_index()

In [ ]:
clickstream_test_w7_new

,session_id,event_total,event_add_promo,event_add_to_cart,event_booking,event_click,event_homepage,event_item_detail,event_promo_page,event_scroll,event_search,event_time_start,event_time_end,is_using_mobile,is_using_web
0,00007415-e0e7-4a33-95e8-5bc749016876,54,0,1,1,18,5.0,8,1,11,9,2022-07-24T13:12:21.038087Z,2022-07-24T13:16:40.038087Z,0,1
1,0000a729-3de3-4613-9022-d12b7fa06e75,4,0,2,1,1,0.0,0,0,0,0,2022-04-02T01:24:07.273818Z,2022-04-09T02:02:54.273818Z,1,0
2,0001afa6-3f2c-4cb9-a118-360831dcb892,11,0,2,1,1,4.0,0,0,1,2,2022-06-18T03:51:48.610015Z,2022-06-28T16:34:01.610015Z,0,1
3,0002ad9d-3c59-4c9c-b6dc-ae6a561ef718,4,0,1,1,0,1.0,0,0,1,0,2022-04-29T17:01:59.008860Z,2022-04-30T07:06:55.008860Z,0,1
4,0003ff81-e79d-4d8a-8dde-bd76727f6a81,20,0,1,1,5,2.0,2,2,7,0,2022-05-09T21:25:04.667324Z,2022-05-13T19:42:33.667324Z,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118719,fffe23af-9ec7-43cb-9f86-550bb58466bc,12,1,1,1,4,3.0,1,1,0,0,2022-05-06T20:31:05.342560Z,2022-05-11T07:23:55.342560Z,1,0
118720,fffe5ead-3b3d-442b-9b1d-99c91412e8f5,10,1,3,1,0,1.0,0,1,1,2,2022-05-18T15:42:44.951998Z,2022-06-06T16:36:53.951998Z,1,0
118721,fffea676-ae7b-48f0-9266-db56eaf9755c,9,1,1,1,2,1.0,0,1,1,1,2022-07-18T04:28:16.204042Z,2022-07-18T09:29:33.204042Z,0,1
118722,fffedbf4-5929-4801-a684-5e50ab05c999,10,1,1,1,3,1.0,0,0,3,0,2022-07-08T15:21:54.697105Z,2022-07-09T00:01:42.697105Z,1,0


### Add New Column (Session Duration Minutes)

In [ ]:
# make 'session_duration_minutes' column (train)
clickstream_train_w1_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w1_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w1_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_train_w2_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w2_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w2_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_train_w3_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w3_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w3_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_train_w4_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w4_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w4_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_train_w5_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w5_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w5_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_train_w6_new['session_duration_minutes'] = (pd.to_datetime(clickstream_train_w6_new.event_time_end) -
                                                     pd.to_datetime(clickstream_train_w6_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

# make 'session_duration_minutes' column (test)
clickstream_test_w1_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w1_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w1_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w2_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w2_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w2_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w3_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w3_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w3_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w4_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w4_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w4_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w5_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w5_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w5_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w6_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w6_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w6_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

clickstream_test_w7_new['session_duration_minutes'] = (pd.to_datetime(clickstream_test_w7_new.event_time_end) -
                                                     pd.to_datetime(clickstream_test_w7_new.event_time_start)
                                                     ).apply(lambda x: x.total_seconds()/60).astype('int64')

## Transaction

#### Drop Unwanted Columns

In [ ]:
transaction.drop(columns=['booking_id', 'product_metadata', 'shipment_location_lat', 'shipment_location_long'], inplace=True)

### Features Engineering

In [ ]:
# make 'shipment_duration" column
transaction['shipment_duration'] = pd.to_datetime(transaction.shipment_date_limit) - pd.to_datetime(transaction.created_at)
transaction['shipment_duration'] = pd.to_numeric(transaction['shipment_duration'].dt.days)
transaction.head()

,created_at,customer_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,total_amount,shipment_duration
0,2018-07-29T15:22:01.458193Z,5868,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,199832,4
1,2018-07-30T12:40:22.365620Z,4774,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,155526,3
2,2018-09-15T11:51:17.365620Z,4774,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,550696,2
3,2018-11-01T11:23:48.365620Z,4774,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,271012,4
4,2018-12-18T11:20:30.365620Z,4774,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,198753,5


In [ ]:
transaction.drop(columns=['shipment_date_limit'], inplace=True)

In [ ]:
# turn 'promo_code' column to 'using_promo' by turn i
transaction['using_promo']=transaction['promo_code'].apply(lambda x : 1 if pd.notna(x) else 0)

In [ ]:
transaction.drop(columns=['promo_code'], inplace=True)

In [ ]:
transaction.head()

,created_at,customer_id,session_id,payment_method,payment_status,promo_amount,shipment_fee,total_amount,shipment_duration,using_promo
0,2018-07-29T15:22:01.458193Z,5868,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,10000,199832,4,1
1,2018-07-30T12:40:22.365620Z,4774,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,10000,155526,3,0
2,2018-09-15T11:51:17.365620Z,4774,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,10000,550696,2,0
3,2018-11-01T11:23:48.365620Z,4774,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,0,271012,4,0
4,2018-12-18T11:20:30.365620Z,4774,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,0,198753,5,0


### One Hot Encoding

In [ ]:
col_names = ['payment_method','payment_status']
one_hot_trans = pd.get_dummies(transaction[col_names], columns=col_names)
one_hot_trans.head()

,payment_method_Credit Card,payment_method_Debit Card,payment_method_Gopay,payment_method_LinkAja,payment_method_OVO,payment_status_Failed,payment_status_Success
0,0,1,0,0,0,0,1
1,1,0,0,0,0,0,1
2,0,0,0,0,1,0,1
3,1,0,0,0,0,0,1
4,1,0,0,0,0,0,1


In [ ]:
transaction = pd.concat([transaction, one_hot_trans], axis=1)
transaction.head()

,created_at,customer_id,session_id,payment_method,payment_status,promo_amount,shipment_fee,total_amount,shipment_duration,using_promo,payment_method_Credit Card,payment_method_Debit Card,payment_method_Gopay,payment_method_LinkAja,payment_method_OVO,payment_status_Failed,payment_status_Success
0,2018-07-29T15:22:01.458193Z,5868,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,10000,199832,4,1,0,1,0,0,0,0,1
1,2018-07-30T12:40:22.365620Z,4774,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,10000,155526,3,0,1,0,0,0,0,0,1
2,2018-09-15T11:51:17.365620Z,4774,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,10000,550696,2,0,0,0,0,0,1,0,1
3,2018-11-01T11:23:48.365620Z,4774,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,0,271012,4,0,1,0,0,0,0,0,1
4,2018-12-18T11:20:30.365620Z,4774,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,0,198753,5,0,1,0,0,0,0,0,1


In [ ]:
transaction.drop(columns=['payment_method','payment_status'], inplace=True)
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852584 entries, 0 to 852583
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   created_at                  852584 non-null  object
 1   customer_id                 852584 non-null  int64 
 2   session_id                  852584 non-null  object
 3   promo_amount                852584 non-null  int64 
 4   shipment_fee                852584 non-null  int64 
 5   total_amount                852584 non-null  int64 
 6   shipment_duration           852584 non-null  int64 
 7   using_promo                 852584 non-null  int64 
 8   payment_method_Credit Card  852584 non-null  uint8 
 9   payment_method_Debit Card   852584 non-null  uint8 
 10  payment_method_Gopay        852584 non-null  uint8 
 11  payment_method_LinkAja      852584 non-null  uint8 
 12  payment_method_OVO          852584 non-null  uint8 
 13  payment_status_Failed       8

### Split Dataset

#### Window #1 (1 Apr 21 - 31 Sep 21)

##### Data Train

In [ ]:
transaction_train_w1 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2021-08-01T00:49:59.408424Z'))]

##### Data Test

In [ ]:
transaction_test_w1 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-08-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2021-10-01T00:49:59.408424Z'))]

#### Window #2 (1 Jun 21 - 31 Nov 21)

##### Data Train

In [ ]:
transaction_train_w2 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-06-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2021-10-01T00:49:59.408424Z'))]


##### Data Test

In [ ]:
transaction_test_w2 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-10-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2021-12-01T00:49:59.408424Z'))]

#### Window #3 (1 Aug 21 - 31 Jan 22)

##### Data Train

In [ ]:
transaction_train_w3 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-08-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2021-12-01T00:49:59.408424Z'))]

##### Data Test

In [ ]:
transaction_test_w3 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-12-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-02-01T00:49:59.408424Z'))]

#### Windows #4 (1 Okt 21 - 31 Mar 22)

##### Data Train

In [ ]:
transaction_train_w4 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-10-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-02-01T00:49:59.408424Z'))]

##### Data Test

In [ ]:
transaction_test_w4 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2022-02-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-04-01T00:49:59.408424Z'))]

#### Window #5 (1 Des 21 - 31 Mei 22)

##### Data Train

In [ ]:
transaction_train_w5 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2021-12-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-04-01T00:49:59.408424Z'))]

##### Data Test

In [ ]:
transaction_test_w5 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2022-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-06-01T00:49:59.408424Z'))]

#### Window #6 (1 Feb 22 - 1 Aug 22)

##### Data Train

In [ ]:
transaction_train_w6 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2022-02-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-06-01T00:49:59.408424Z'))]

##### Data Test

In [ ]:
transaction_test_w6 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2022-06-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-08-01T00:49:59.408424Z'))]

#### Window #7 (1 Apr 22 - 1 Aug 22)

##### Data Test

In [ ]:
transaction_test_w7 = transaction[(pd.to_datetime(transaction['created_at']) >= pd.to_datetime('2022-04-01T00:49:59.408424Z')) &
                               (pd.to_datetime(transaction['created_at']) < pd.to_datetime('2022-08-01T00:49:59.408424Z'))]

## Customer

### Feature Engineering

In [ ]:
customer['birthdate']=pd.to_datetime(customer['birthdate'])
customer['first_join_date']=pd.to_datetime(customer['first_join_date'])
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   customer_id         100000 non-null  int64         
 1   first_name          100000 non-null  object        
 2   last_name           100000 non-null  object        
 3   username            100000 non-null  object        
 4   email               100000 non-null  object        
 5   gender              100000 non-null  object        
 6   birthdate           100000 non-null  datetime64[ns]
 7   device_type         100000 non-null  object        
 8   device_id           100000 non-null  object        
 9   device_version      100000 non-null  object        
 10  home_location_lat   100000 non-null  float64       
 11  home_location_long  100000 non-null  float64       
 12  home_location       100000 non-null  object        
 13  home_country        100000 non

In [ ]:
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

def calculate_join_date(join):
    date2 = date.today()
    r = relativedelta.relativedelta(date2, join)
    # return r.years #Tahun
    return r.years * 12 + r.months #Bulan

In [ ]:
customer['customer_age_years'] = customer['birthdate'].apply(lambda birth: calculate_age(birth))
customer['account_age_months'] = customer['first_join_date'].apply(lambda joined: calculate_join_date(joined))
customer.head()

,customer_id,first_name,last_name,username,email,gender,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date,customer_age_years,account_age_months
0,2870,Lala,Maryati,671a0865-ac4e-4dc4-9c4f-c286a1176f7e,671a0865_ac4e_4dc4_9c4f_c286a1176f7e@startupca...,F,1996-06-14,iOS,c9c0de76-0a6c-4ac2-843f-65264ab9fe63,iPhone; CPU iPhone OS 14_2_1 like Mac OS X,-1.043345,101.360523,Sumatera Barat,Indonesia,2019-07-21,26,40
1,8193,Maimunah,Laksmiwati,83be2ba7-8133-48a4-bbcb-b46a2762473f,83be2ba7_8133_48a4_bbcb_b46a2762473f@zakyfound...,F,1993-08-16,Android,fb331c3d-f42e-40fe-afe2-b4b73a8a6e25,Android 2.2.1,-6.212489,106.818850,Jakarta Raya,Indonesia,2017-07-16,29,64
2,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,-8.631607,116.428436,Nusa Tenggara Barat,Indonesia,2020-08-23,33,27
3,88813,Cahyadi,Maheswara,df797edf-b465-4a80-973b-9fbb612260c2,df797edf_b465_4a80_973b_9fbb612260c2@zakyfound...,M,1991-01-05,iOS,f4c18515-c5be-419f-8142-f037be47c9cd,iPad; CPU iPad OS 14_2 like Mac OS X,1.299332,115.774934,Kalimantan Timur,Indonesia,2021-10-03,31,14
4,82542,Irnanto,Wijaya,36ab08e1-03de-42a8-9e3b-59528c798824,36ab08e1_03de_42a8_9e3b_59528c798824@startupca...,M,2000-07-15,iOS,e46e4c36-4630-4736-8fcf-663db29ca3b0,iPhone; CPU iPhone OS 10_3_3 like Mac OS X,-2.980807,114.924675,Kalimantan Selatan,Indonesia,2021-04-11,22,19


### One Hot Encoding

In [ ]:
one_hot = pd.get_dummies(customer['gender'], columns='gender')
customer = pd.concat([customer, one_hot], axis=1)

In [ ]:
customer.rename(columns={'F': 'gender_female', 'M': 'gender_male'}, inplace=True)

### Drop Unwanted Columns

In [ ]:
customer.drop(columns=['first_name', 'last_name', 'username', 'email', 'gender',
                       'birthdate', 'device_type', 'device_id', 'device_version',
                       'home_location_lat', 'home_location_long', 'home_location',
                       'home_country', 'first_join_date'], inplace=True)

In [ ]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   customer_id         100000 non-null  int64
 1   customer_age_years  100000 non-null  int64
 2   account_age_months  100000 non-null  int64
 3   gender_female       100000 non-null  uint8
 4   gender_male         100000 non-null  uint8
dtypes: int64(3), uint8(2)
memory usage: 2.5 MB


## Data Merging

### Clickstream & Transaction

#### Data Train & Data Test

In [ ]:
# merge clickstream to transaction (train)
clickstream_transaction_train_w1 = clickstream_train_w1_new.merge(transaction_train_w1, how='inner', on='session_id')

clickstream_transaction_train_w2 = clickstream_train_w2_new.merge(transaction_train_w2, how='inner', on='session_id')

clickstream_transaction_train_w3 = clickstream_train_w3_new.merge(transaction_train_w3, how='inner', on='session_id')

clickstream_transaction_train_w4 = clickstream_train_w4_new.merge(transaction_train_w4, how='inner', on='session_id')

clickstream_transaction_train_w5 = clickstream_train_w5_new.merge(transaction_train_w5, how='inner', on='session_id')

clickstream_transaction_train_w6 = clickstream_train_w6_new.merge(transaction_train_w6, how='inner', on='session_id')

# merge clickstream to transaction (test)
clickstream_transaction_test_w1 = clickstream_test_w1_new.merge(transaction_test_w1, how='inner', on='session_id')

clickstream_transaction_test_w2 = clickstream_test_w2_new.merge(transaction_test_w2, how='inner', on='session_id')

clickstream_transaction_test_w3 = clickstream_test_w3_new.merge(transaction_test_w3, how='inner', on='session_id')

clickstream_transaction_test_w4 = clickstream_test_w4_new.merge(transaction_test_w4, how='inner', on='session_id')

clickstream_transaction_test_w5 = clickstream_test_w5_new.merge(transaction_test_w5, how='inner', on='session_id')

clickstream_transaction_test_w6 = clickstream_test_w6_new.merge(transaction_test_w6, how='inner', on='session_id')

clickstream_transaction_test_w7 = clickstream_test_w7_new.merge(transaction_test_w6, how='inner', on='session_id')

#### Drop Unwanted Columns

In [ ]:
# drop unwanted column (train)
clickstream_transaction_train_w1.drop(columns=['session_id'], inplace=True)
clickstream_transaction_train_w2.drop(columns=['session_id'], inplace=True)
clickstream_transaction_train_w3.drop(columns=['session_id'], inplace=True)
clickstream_transaction_train_w4.drop(columns=['session_id'], inplace=True)
clickstream_transaction_train_w5.drop(columns=['session_id'], inplace=True)
clickstream_transaction_train_w6.drop(columns=['session_id'], inplace=True)

# drop unwanted column (test)
clickstream_transaction_test_w1.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w2.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w3.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w4.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w5.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w6.drop(columns=['session_id'], inplace=True)
clickstream_transaction_test_w7.drop(columns=['session_id'], inplace=True)

### Clickstream & Transaction & Customer

#### Data Train & Data Test

In [ ]:
# merge clickstream, transaction, customer (train)
df_train_w1 = customer.merge(clickstream_transaction_train_w1, how='inner', on='customer_id')
df_train_w2 = customer.merge(clickstream_transaction_train_w2, how='inner', on='customer_id')
df_train_w3 = customer.merge(clickstream_transaction_train_w3, how='inner', on='customer_id')
df_train_w4 = customer.merge(clickstream_transaction_train_w4, how='inner', on='customer_id')
df_train_w5 = customer.merge(clickstream_transaction_train_w5, how='inner', on='customer_id')
df_train_w6 = customer.merge(clickstream_transaction_train_w6, how='inner', on='customer_id')

# merge clickstream, transaction, customer (test)
df_test_w1 = customer.merge(clickstream_transaction_test_w1, how='inner', on='customer_id')
df_test_w2 = customer.merge(clickstream_transaction_test_w2, how='inner', on='customer_id')
df_test_w3 = customer.merge(clickstream_transaction_test_w3, how='inner', on='customer_id')
df_test_w4 = customer.merge(clickstream_transaction_test_w4, how='inner', on='customer_id')
df_test_w5 = customer.merge(clickstream_transaction_test_w5, how='inner', on='customer_id')
df_test_w6 = customer.merge(clickstream_transaction_test_w6, how='inner', on='customer_id')
df_test_w7 = customer.merge(clickstream_transaction_test_w7, how='inner', on='customer_id')

### Feature Engineering

#### Window #1 (1 Apr 21 - 31 Sep 21)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w1_new=df_train_w1.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w1_new=df_test_w1.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Window #2 (1 Jun 21 - 31 Nov 21)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w2_new=df_train_w2.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w2_new=df_test_w2.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Window #3 (1 Aug 21 - 31 Jan 22)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w3_new=df_train_w3.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w3_new=df_test_w3.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Windows #4 (1 Okt 21 - 31 Mar 22)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w4_new=df_train_w4.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w4_new=df_test_w4.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Window #5 (1 Des 21 - 31 Mei 22)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w5_new=df_train_w5.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w5_new=df_test_w5.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Window #6 (1 Feb 22 - 1 Aug 22)

##### Data Train

In [ ]:
# Group by customer_id & Aggregating (train)
df_train_w6_new=df_train_w6.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w6_new=df_test_w6.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

#### Window #7 (1 Apr 22 - 1 Aug 22)

##### Data Test

In [ ]:
# Group by customer_id & Aggregating (test)
df_test_w7_new=df_test_w7.groupby('customer_id').agg(
    customer_age_years=pd.NamedAgg(column='customer_age_years', aggfunc='max'),
    account_age_months=pd.NamedAgg(column='account_age_months', aggfunc='max'),
    gender_female=pd.NamedAgg(column='gender_female', aggfunc='max'),
    gender_male=pd.NamedAgg(column='gender_male', aggfunc='max'),
    event_total=pd.NamedAgg(column='event_total', aggfunc='sum'),
    event_add_promo_total=pd.NamedAgg(column='event_add_promo', aggfunc='sum'),
    event_add_to_cart_total=pd.NamedAgg(column='event_add_to_cart', aggfunc='sum'),
    event_booking_total=pd.NamedAgg(column='event_booking', aggfunc='sum'),
    event_click_total=pd.NamedAgg(column='event_click', aggfunc='sum'),
    event_homepage_total=pd.NamedAgg(column='event_homepage', aggfunc='sum'),
    event_item_detail_total=pd.NamedAgg(column='event_item_detail', aggfunc='sum'),
    event_promo_page_total=pd.NamedAgg(column='event_promo_page', aggfunc='sum'),
    event_scroll_total=pd.NamedAgg(column='event_scroll', aggfunc='sum'),
    event_search_total=pd.NamedAgg(column='event_search', aggfunc='sum'),
    promo_amount_total=pd.NamedAgg(column='promo_amount', aggfunc='sum'),
    promo_amount_avg=pd.NamedAgg(column='promo_amount', aggfunc='mean'),
    shipment_fee_total=pd.NamedAgg(column='shipment_fee', aggfunc='sum'),
    shipment_fee_avg=pd.NamedAgg(column='shipment_fee', aggfunc='mean'),
    total_amount_product_total=pd.NamedAgg(column='total_amount', aggfunc='sum'),
    total_amount_product_avg=pd.NamedAgg(column='total_amount', aggfunc='mean'),
    shipment_duration_avg=pd.NamedAgg(column='shipment_duration', aggfunc='mean'),
    using_promo_total=pd.NamedAgg(column='using_promo', aggfunc='sum'),
    payment_creditcard_total=pd.NamedAgg(column='payment_method_Credit Card', aggfunc='sum'),
    payment_debitcard_total=pd.NamedAgg(column='payment_method_Debit Card', aggfunc='sum'),
    payment_gopay_total=pd.NamedAgg(column='payment_method_Gopay', aggfunc='sum'),
    payment_linkaja_total=pd.NamedAgg(column='payment_method_LinkAja', aggfunc='sum'),
    payment_ovo_total=pd.NamedAgg(column='payment_method_OVO', aggfunc='sum'),
    payment_failed_total=pd.NamedAgg(column='payment_status_Failed', aggfunc='sum'),
    payment_success_total=pd.NamedAgg(column='payment_status_Success', aggfunc='sum'),
    session_duration_minutes_total=pd.NamedAgg(column='session_duration_minutes', aggfunc='sum'),
    session_duration_minutes_avg=pd.NamedAgg(column='session_duration_minutes', aggfunc='mean'),
    using_mobile_total=pd.NamedAgg(column='is_using_mobile', aggfunc='sum'),
    using_web_total=pd.NamedAgg(column='is_using_web', aggfunc='sum'),
    recent_transaction=pd.NamedAgg(column='created_at', aggfunc='max')
    ).reset_index()

### Add New Column (Transaction Recency Days)

In [ ]:
# Make recency columm (train)
df_train_w1_new['transaction_recency_days'] = pd.to_datetime('2021-10-01T00:49:59.408424Z') - pd.to_datetime(df_train_w1_new.recent_transaction)
df_train_w1_new['transaction_recency_days'] = pd.to_numeric(df_train_w1_new['transaction_recency_days'].dt.days)
df_train_w1_new.drop(columns=['recent_transaction'], inplace=True)

df_train_w2_new['transaction_recency_days'] = pd.to_datetime('2021-12-01T00:49:59.408424Z') - pd.to_datetime(df_train_w2_new.recent_transaction)
df_train_w2_new['transaction_recency_days'] = pd.to_numeric(df_train_w2_new['transaction_recency_days'].dt.days)
df_train_w2_new.drop(columns=['recent_transaction'], inplace=True)

df_train_w3_new['transaction_recency_days'] = pd.to_datetime('2022-02-01T00:49:59.408424Z') - pd.to_datetime(df_train_w3_new.recent_transaction)
df_train_w3_new['transaction_recency_days'] = pd.to_numeric(df_train_w3_new['transaction_recency_days'].dt.days)
df_train_w3_new.drop(columns=['recent_transaction'], inplace=True)

df_train_w4_new['transaction_recency_days'] = pd.to_datetime('2022-04-01T00:49:59.408424Z') - pd.to_datetime(df_train_w4_new.recent_transaction)
df_train_w4_new['transaction_recency_days'] = pd.to_numeric(df_train_w4_new['transaction_recency_days'].dt.days)
df_train_w4_new.drop(columns=['recent_transaction'], inplace=True)

df_train_w5_new['transaction_recency_days'] = pd.to_datetime('2022-06-01T00:49:59.408424Z') - pd.to_datetime(df_train_w5_new.recent_transaction)
df_train_w5_new['transaction_recency_days'] = pd.to_numeric(df_train_w5_new['transaction_recency_days'].dt.days)
df_train_w5_new.drop(columns=['recent_transaction'], inplace=True)

df_train_w6_new['transaction_recency_days'] = pd.to_datetime('2022-08-01T00:49:59.408424Z') - pd.to_datetime(df_train_w6_new.recent_transaction)
df_train_w6_new['transaction_recency_days'] = pd.to_numeric(df_train_w6_new['transaction_recency_days'].dt.days)
df_train_w6_new.drop(columns=['recent_transaction'], inplace=True)

# Make recency columm (test)
df_test_w1_new['transaction_recency_days'] = pd.to_datetime('2021-10-01T00:49:59.408424Z') - pd.to_datetime(df_test_w1_new.recent_transaction)
df_test_w1_new['transaction_recency_days'] = pd.to_numeric(df_test_w1_new['transaction_recency_days'].dt.days)
df_test_w1_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w2_new['transaction_recency_days'] = pd.to_datetime('2021-12-01T00:49:59.408424Z') - pd.to_datetime(df_test_w2_new.recent_transaction)
df_test_w2_new['transaction_recency_days'] = pd.to_numeric(df_test_w2_new['transaction_recency_days'].dt.days)
df_test_w2_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w3_new['transaction_recency_days'] = pd.to_datetime('2022-02-01T00:49:59.408424Z') - pd.to_datetime(df_test_w3_new.recent_transaction)
df_test_w3_new['transaction_recency_days'] = pd.to_numeric(df_test_w3_new['transaction_recency_days'].dt.days)
df_test_w3_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w4_new['transaction_recency_days'] = pd.to_datetime('2022-04-01T00:49:59.408424Z') - pd.to_datetime(df_test_w4_new.recent_transaction)
df_test_w4_new['transaction_recency_days'] = pd.to_numeric(df_test_w4_new['transaction_recency_days'].dt.days)
df_test_w4_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w5_new['transaction_recency_days'] = pd.to_datetime('2022-06-01T00:49:59.408424Z') - pd.to_datetime(df_test_w5_new.recent_transaction)
df_test_w5_new['transaction_recency_days'] = pd.to_numeric(df_test_w5_new['transaction_recency_days'].dt.days)
df_test_w5_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w6_new['transaction_recency_days'] = pd.to_datetime('2022-08-01T00:49:59.408424Z') - pd.to_datetime(df_test_w6_new.recent_transaction)
df_test_w6_new['transaction_recency_days'] = pd.to_numeric(df_test_w6_new['transaction_recency_days'].dt.days)
df_test_w6_new.drop(columns=['recent_transaction'], inplace=True)

df_test_w7_new['transaction_recency_days'] = pd.to_datetime('2022-08-01T00:49:59.408424Z') - pd.to_datetime(df_test_w7_new.recent_transaction)
df_test_w7_new['transaction_recency_days'] = pd.to_numeric(df_test_w7_new['transaction_recency_days'].dt.days)
df_test_w7_new.drop(columns=['recent_transaction'], inplace=True)

In [ ]:
df_train_w1_new.sort_values(by='transaction_recency_days', ascending=False)

,customer_id,customer_age_years,account_age_months,gender_female,gender_male,event_total,event_add_promo_total,event_add_to_cart_total,event_booking_total,event_click_total,...,payment_gopay_total,payment_linkaja_total,payment_ovo_total,payment_failed_total,payment_success_total,session_duration_minutes_total,session_duration_minutes_avg,using_mobile_total,using_web_total,transaction_recency_days
13257,50099,21,52,1,0,1,0,0,1,0,...,1,0,0,0,1,0,0.000000,1,0,182
13801,52280,34,44,1,0,1,0,0,1,0,...,1,0,0,0,1,0,0.000000,0,1,182
6921,25851,23,20,1,0,11,0,4,1,3,...,0,0,0,0,1,52,52.000000,1,0,182
22157,84609,19,36,1,0,1,0,0,1,0,...,1,0,0,0,1,0,0.000000,1,0,182
3960,14459,39,46,1,0,1,0,0,1,0,...,1,0,0,1,0,0,0.000000,1,0,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4832,17714,27,67,1,0,277,6,20,16,63,...,2,3,0,1,15,58439,3652.437500,14,2,61
11493,43253,23,28,1,0,207,2,8,8,62,...,7,0,1,0,8,107698,13462.250000,8,0,61
2020,7386,27,34,1,0,445,10,114,41,66,...,3,1,3,3,38,36140,881.463415,37,4,61
4837,17737,24,58,1,0,99,3,6,6,23,...,0,0,1,1,5,64751,10791.833333,4,2,61


## Make Label

### Label Data Train

In [ ]:
label_w1 = pd.DataFrame(df_test_w1_new['customer_id'])
label_w1['is_exist'] = 1

label_w2 = pd.DataFrame(df_test_w2_new['customer_id'])
label_w2['is_exist'] = 1

label_w3 = pd.DataFrame(df_test_w3_new['customer_id'])
label_w3['is_exist'] = 1

label_w4 = pd.DataFrame(df_test_w4_new['customer_id'])
label_w4['is_exist'] = 1

label_w5 = pd.DataFrame(df_test_w5_new['customer_id'])
label_w5['is_exist'] = 1

label_w6 = pd.DataFrame(df_test_w6_new['customer_id'])
label_w6['is_exist'] = 1

In [ ]:
df_train_w1_label = df_train_w1_new.merge(label_w1, how='left', on='customer_id')
df_train_w1_label.info()

df_train_w2_label = df_train_w2_new.merge(label_w2, how='left', on='customer_id')
df_train_w2_label.info()

df_train_w3_label = df_train_w3_new.merge(label_w3, how='left', on='customer_id')
df_train_w3_label.info()

df_train_w4_label = df_train_w4_new.merge(label_w4, how='left', on='customer_id')
df_train_w4_label.info()

df_train_w5_label = df_train_w5_new.merge(label_w5, how='left', on='customer_id')
df_train_w5_label.info()

df_train_w6_label = df_train_w6_new.merge(label_w6, how='left', on='customer_id')
df_train_w6_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26204 entries, 0 to 26203
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   customer_id                     26204 non-null  int64  
 1   customer_age_years              26204 non-null  int64  
 2   account_age_months              26204 non-null  int64  
 3   gender_female                   26204 non-null  uint8  
 4   gender_male                     26204 non-null  uint8  
 5   event_total                     26204 non-null  int64  
 6   event_add_promo_total           26204 non-null  uint8  
 7   event_add_to_cart_total         26204 non-null  uint8  
 8   event_booking_total             26204 non-null  uint8  
 9   event_click_total               26204 non-null  uint8  
 10  event_homepage_total            26204 non-null  float64
 11  event_item_detail_total         26204 non-null  uint8  
 12  event_promo_page_total          

In [ ]:
df_train_w1_label['is_churn']=df_train_w1_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

df_train_w2_label['is_churn']=df_train_w2_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

df_train_w3_label['is_churn']=df_train_w3_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

df_train_w4_label['is_churn']=df_train_w4_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

df_train_w5_label['is_churn']=df_train_w5_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

df_train_w6_label['is_churn']=df_train_w6_label['is_exist'].apply(lambda x: 0 if x==1 else 1)

In [ ]:
df_train_w1_label.drop(columns='is_exist', inplace=True)
df_train_w2_label.drop(columns='is_exist', inplace=True)
df_train_w3_label.drop(columns='is_exist', inplace=True)
df_train_w4_label.drop(columns='is_exist', inplace=True)
df_train_w5_label.drop(columns='is_exist', inplace=True)
df_train_w6_label.drop(columns='is_exist', inplace=True)

In [ ]:
df_train_w1_label.info()
df_train_w2_label.info()
df_train_w3_label.info()
df_train_w4_label.info()
df_train_w5_label.info()
df_train_w6_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26204 entries, 0 to 26203
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   customer_id                     26204 non-null  int64  
 1   customer_age_years              26204 non-null  int64  
 2   account_age_months              26204 non-null  int64  
 3   gender_female                   26204 non-null  uint8  
 4   gender_male                     26204 non-null  uint8  
 5   event_total                     26204 non-null  int64  
 6   event_add_promo_total           26204 non-null  uint8  
 7   event_add_to_cart_total         26204 non-null  uint8  
 8   event_booking_total             26204 non-null  uint8  
 9   event_click_total               26204 non-null  uint8  
 10  event_homepage_total            26204 non-null  float64
 11  event_item_detail_total         26204 non-null  uint8  
 12  event_promo_page_total          

In [ ]:
df_train_w2_label[df_train_w2_label['transaction_recency_days'] < 100].sort_values(by='transaction_recency_days', ascending=False).head(50)

,customer_id,customer_age_years,account_age_months,gender_female,gender_male,event_total,event_add_promo_total,event_add_to_cart_total,event_booking_total,event_click_total,...,payment_linkaja_total,payment_ovo_total,payment_failed_total,payment_success_total,session_duration_minutes_total,session_duration_minutes_avg,using_mobile_total,using_web_total,transaction_recency_days,is_churn
7800,27625,24,15,1,0,14,0,2,1,4,...,0,0,0,1,1,1.000000,1,0,99,0
22399,80266,27,45,1,0,14,1,2,1,5,...,0,0,0,1,197,197.000000,1,0,99,0
3842,13156,23,74,1,0,6,0,2,1,1,...,0,0,0,1,47245,47245.000000,1,0,99,1
27541,98630,19,23,1,0,10,1,4,1,1,...,0,0,0,1,72262,72262.000000,1,0,99,0
10477,36982,32,54,0,1,13,1,4,1,6,...,0,1,0,1,20292,20292.000000,1,0,99,0
27547,98660,13,73,0,1,18,0,6,1,3,...,0,0,0,1,5089,5089.000000,1,0,99,0
1734,5945,21,18,0,1,26,1,7,2,5,...,0,0,0,2,26915,13457.500000,2,0,99,0
22332,80006,21,75,0,1,34,1,2,2,3,...,0,0,0,2,46131,23065.500000,2,0,99,0
14011,49646,32,38,1,0,31,0,8,2,4,...,0,0,0,2,94563,47281.500000,1,1,99,0
23965,86003,38,26,0,1,14,0,3,2,3,...,0,1,0,2,37975,18987.500000,2,0,99,0


# Final Dataset

In [ ]:
df_train_w1_final_model = df_train_w1_label
df_train_w2_final_model = df_train_w2_label
df_train_w3_final_model = df_train_w3_label
df_train_w4_final_model = df_train_w4_label
df_train_w5_final_model = df_train_w5_label
df_train_w6_final_model = df_train_w6_label
df_train_w7_final_model = df_test_w7_new

# Export Final Dataset

In [ ]:
# FP - Window Dataset

# df_train_w1_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w1_model_train.csv', index=False)
# df_train_w2_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w2_model_train.csv', index=False)
# df_train_w3_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w3_model_train.csv', index=False)
# df_train_w4_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w4_model_train.csv', index=False)
# df_train_w5_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w5_model_train.csv', index=False)
# df_train_w6_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w6_model_validation.csv', index=False)
# df_train_w7_final_model.to_csv('/content/drive/MyDrive/StartupCampus/FP - Window Dataset/df_features_w7_model_predict.csv', index=False)